In [1]:
#!pip3 freeze > "../requirements.txt"
#!pip3 install -r "../requirements.txt"  # giving some error
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import copy
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from pins import board_rsconnect
from collections import Counter
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 


def GBQ_data(query_string):
    client = bigquery.Client('turing-230020')
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()

def ets_out_cluster(ets_data, cluster):    
    ets_out = pd.DataFrame(columns=['skill_name', 'num_jobs', 'num_DC', 'DC_per'])
    ind=0
    for sk in skill_names:
        ets_out.at[ind, 'skill_name'] = sk
        num_jobs = 0
        num_DC = 0
        ets_data = ets_data.loc[ets_data['cluster']==cluster].reset_index(drop=True)
        for job in ets_data.index:
            if sk in ets_data.at[job, 'must_have_skill_names_list']:
                num_jobs = num_jobs+1
                num_DC = num_DC + ets_data.at[job, 'developer_chosen']
        ets_out.at[ind, 'num_jobs'] = num_jobs
        ets_out.at[ind, 'num_DC'] = num_DC
        try:
            ets_out.at[ind, 'DC_per'] = round(num_DC/num_jobs,2)
        except ZeroDivisionError:
            ets_out.at[ind, 'DC_per'] = 0
        ind = ind+1        
    ets_out['cluster'] = cluster
    return ets_out

In [2]:
job_mcq_query = """SELECT job_id, 
string_agg(distinct case when job_skill_level_id=1 then cast(skill_id as string) else NULL end order by case when job_skill_level_id=1 then cast(skill_id as string) else NULL end) must_have_skill_ids,
string_agg(distinct case when job_skill_level_id=1 then skill_name else NULL end) must_have_skill_names
from(
SELECT job.job_id, job.skill_id, job.job_skill_level_id, skill.skill_name, ch_name.challenge_name , mcq_id
from `turing-230020.devdb_mirror.ms2_job_skill` job
LEFT join `turing-230020.devdb_mirror.base_all_skills_v4` skill on job.skill_id=skill.id
Left JOIN `turing-230020.devdb_mirror.dv2_skill_mcq` skill_mcq on job.skill_id=skill_mcq.skill_id
left join `turing-230020.devdb_mirror.dv2_challenge` ch_name on skill_mcq.mcq_id=ch_name.challenge_id 
) GROUP by 1
"""
job_mcq = GBQ_data(job_mcq_query)
print(job_mcq.shape)
job_mcq.head(2)

(8121, 3)


,job_id,must_have_skill_ids,must_have_skill_names
0,512,"120,2031","Node.js,React"
1,768,1561,Mobile App Development


## Enterprise single skill how many jobs and DCs for each cluster

In [3]:
ets_data = pd.read_csv('clustered-data/enterprise-cluster-data-2022-08-16.csv', index_col=0)
ets_data = ets_data.merge(job_mcq, how='left', on='job_id')
ets_data = ets_data.loc[ets_data['num_must_have_skills']>0].reset_index(drop=True)
ets_data['cluster'] = ets_data['cluster'].map({1:0, 0:1, 2:2})
#ets_data.groupby('cluster').agg({'job_id':'count', 'developer_chosen':'mean',  'num_must_have_skills':'mean'})
skill_names = []
for i in ets_data.index:
    skill_names.append(ets_data.at[i,'must_have_skill_names'].split(',',-1))
skill_names = [val for sublist in skill_names for val in sublist]
skill_names = [val.strip() for val in skill_names]
skill_names = list(set(skill_names)) 
ets_data['must_have_skill_names_list'] =  ets_data.must_have_skill_names.apply(lambda x:x.split(',', -1))
display(ets_data.head(2))

a0 = ets_out_cluster(ets_data,0)
a1 = ets_out_cluster(ets_data,1)
a2 = ets_out_cluster(ets_data,2)
pd.concat([a0,a1,a2])#.to_csv('ets-skills.csv')

,client_type,client_category,region,role_type,priority,num_skills,num_mcqs,num_must_have_skills,num_nice_have_skills,n_devs_must_have_mcq,n_devs_nice_have_mcq,n_devs_must_have_self,n_devs_nice_have_self,job_value,customer_weekly_hour_engagement,is_deleted,cluster,developer_chosen,job_id,must_have_skill_ids,must_have_skill_names,must_have_skill_names_list
0,existing,1.Platinum,2.Rest of US,NaN,NaN,2,2,1,1,0.0,9808.0,143.0,17893.0,29.2,NaN,0,1,1.0,11,1675,Salesforce Development,[Salesforce Development]
1,existing,1.Platinum,2.Rest of US,Fullstack (BE-heavy),p0,3,4,3,0,199.0,No Nice have skill,3807.0,No Nice have skill,29.2,40.0,0,1,1.0,7825,"114,120,2031","Node.js,React,SQL","[Node.js, React, SQL]"


,skill_name,num_jobs,num_DC,DC_per,cluster
0,Windows,0,0,0,0
1,Microservices,0,0,0,0
2,Oracle Database Administration,0,0,0,0
3,Docker,0,0,0,0
4,Android SDK,0,0,0,0
...,...,...,...,...,...
270,AWS Operations,0,0,0,2
271,C++,12,1,0.08,2
272,Data Pipelines,9,2,0.22,2
273,Database Administration,2,0,0,2


## FSS single skill how many jobs and DCs for each cluster

In [4]:
fss_data = pd.read_csv('clustered-data/fss-cluster-data-2022-08-16.csv', index_col=0)
fss_data = fss_data.merge(job_mcq, how='left', on='job_id')
fss_data = fss_data.loc[(fss_data['num_must_have_skills']>0) & (fss_data['cluster'].isin([0,1,2,3,4,5]))].reset_index(drop=True).copy()
fss_data['cluster'] = np.where(fss_data['cluster']==3, 4, fss_data['cluster'])
fss_data['cluster'] = np.where(fss_data['cluster'].isin([0,2]), 1, fss_data['cluster'])
fss_data['cluster'] = fss_data['cluster'].map({5:0, 1:1, 4:2})
#fss_data.groupby('cluster').agg({'job_id':'count', 'developer_chosen':'mean',  'num_must_have_skills':'mean'})
skill_names = []
for i in fss_data.index:
    skill_names.append(fss_data.at[i,'must_have_skill_names'].split(',',-1))
skill_names = [val for sublist in skill_names for val in sublist]
skill_names = [val.strip() for val in skill_names]
skill_names = list(set(skill_names)) 
fss_data['must_have_skill_names_list'] =  fss_data.must_have_skill_names.apply(lambda x:x.split(',', -1))
display(fss_data.head(2))


a0 = ets_out_cluster(fss_data,0)
a1 = ets_out_cluster(fss_data,1)
a2 = ets_out_cluster(fss_data,2)
pd.concat([a0,a1,a2]).to_csv('fss-skills.csv')

,client_type,client_category,region,role_type,priority,num_skills,num_mcqs,num_must_have_skills,num_nice_have_skills,n_devs_must_have_mcq,n_devs_nice_have_mcq,n_devs_must_have_self,n_devs_nice_have_self,job_value,customer_weekly_hour_engagement,is_deleted,cluster,target,job_id,developer_chosen,must_have_skill_names,must_have_skill_names_list
0,existing,4.Bronze,2.Rest of US,NaN,NaN,2,3,2,0,114.0,No Nice have skill,7232.0,No Nice have skill,3.9,NaN,0,0,1.0,55,1.0,"PHP,JavaScript","[PHP, JavaScript]"
1,new,4.Bronze,2.Rest of US,NaN,NaN,2,3,2,0,1410.0,No Nice have skill,10025.0,No Nice have skill,7.4,NaN,0,2,1.0,3,1.0,"React,Node.js","[React, Node.js]"


## Skill combination analysis
- How many jobs we have per skill per cluster and what is DC rate.


In [3]:
ets_data = pd.read_csv('clustered-data/enterprise-cluster-data-2022-08-16.csv', index_col=0)
ets_data = ets_data.merge(job_mcq, how='left', on='job_id')
ets_data = ets_data.loc[ets_data['num_must_have_skills']>0].reset_index(drop=True)
ets_data['cluster'] = ets_data['cluster'].map({1:0, 0:1, 2:2})

ets_data['must_have_skill_names_list'] = ets_data['must_have_skill_names'].apply(lambda x:x.split(',', -1))
ets_data['must_have_skill_names_list_sort'] = ''
for i in ets_data.index:
    ets_data.at[i , 'must_have_skill_names_list_sort'] = sorted(ets_data.at[i , 'must_have_skill_names_list'])


aa = ets_data.groupby(['cluster', 'must_have_skill_ids']).agg({'job_id':'count', 'developer_chosen':'mean'}).reset_index()

temp_df = ets_data[['must_have_skill_ids', 'must_have_skill_names_list_sort']]
temp_df = temp_df.drop_duplicates(subset = ['must_have_skill_ids'])
aa.merge(temp_df, on='must_have_skill_ids', how='left')

NameError: name 'pd' is not defined